# Evaluate to performance of model(s) according to hyperparameters

In [2]:
import os
import torch
import torch.nn.functional as F
import sklearn.metrics as skm
import numpy as np
import pandas as pd

from helpers.mnist_loader import loadMNISTDatasets, getMNISTLoaders
import helpers.evaluation as evaluation

import models.MLP as MLPs
import models.CNN as CNNs

matplotlib.rcParams['figure.figsize'] = (10, 5)

train_dataset, val_dataset, holdback_dataset = loadMNISTDatasets()
_,_, holdback_loader = getMNISTLoaders([train_dataset, val_dataset, holdback_dataset],batch_size=1000)

device = 'cuda' if torch.cuda.is_available() else 'cpu';

/home/william/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


## Load the results

In [3]:
results_file = "two_test_60epochs"

if not os.path.exists(f'results/{results_file}.csv'):
    print('ERROR missing training file! Try retraining to produce the needed file!')
else:
    model_hp_log = pd.read_csv(f'results/{results_file}.csv', index_col=0, usecols=[0,2,3,4,5,6,7,8])
    

In [4]:
model_hp_log.head()

,epoch,loss,val_loss,accuracy,val_accuracy,learning_rate,batch_size
0,1,0.2836,0.1333,0.9110,0.9584,0.2,64
1,2,0.1876,0.1151,0.9412,0.9643,0.2,64
2,3,0.1437,0.1047,0.9551,0.9681,0.2,64
3,4,0.1178,0.1013,0.9631,0.9697,0.2,64
4,5,0.1000,0.0986,0.9687,0.9709,0.2,64


In [5]:
nb_epochs = 60

model_hp_log[model_hp_log.epoch == nb_epochs].sort_values(by=['val_accuracy'], ascending=False)

,epoch,loss,val_loss,accuracy,val_accuracy,learning_rate,batch_size
59,60,0.0091,0.0975,0.9972,0.9814,0.20,64
119,60,0.0205,0.0920,0.9943,0.9777,0.20,256
239,60,0.1383,0.1570,0.9622,0.9547,0.02,256
179,60,0.2698,0.3242,0.8907,0.8772,0.02,64


### Choosing best models

with the above table, we can choose the hyperparameters that yield the best performance.

In [40]:
best_lr = .07
best_bs = 64

best_one = model_hp_log[
            (model_hp_log.learning_rate == best_lr) & 
            (model_hp_log.batch_size == best_bs)
           ]
best_one

,epoch,loss,val_loss,accuracy,val_accuracy,learning_rate,batch_size
320,1,0.3718,0.1841,0.8903,0.9437,0.07,64
321,2,0.2560,0.1564,0.9247,0.9536,0.07,64
322,3,0.2028,0.1391,0.9404,0.9584,0.07,64
323,4,0.1704,0.1276,0.9500,0.9617,0.07,64
324,5,0.1477,0.1192,0.9567,0.9642,0.07,64
...,...,...,...,...,...,...,...
395,76,0.0141,0.0733,0.9965,0.9799,0.07,64
396,77,0.0140,0.0734,0.9965,0.9800,0.07,64
397,78,0.0138,0.0734,0.9966,0.9800,0.07,64
398,79,0.0136,0.0734,0.9966,0.9800,0.07,64


In [41]:
best_name= f'results/{results_file}_best.csv'

if os.path.exists(best_name):
    print('ERROR Best model file already exists!')
else:
    best_one.to_csv(best_name)

### Checking accuracy on the holdback set

In [30]:
import glob
model_type = "MLPTwo"
models = []
for f in glob.glob(f"./models/saved/model_{model_type}_*"):
    models.append(f.split("/")[3])
    print(models[-1])

model_MLPTwo_learning_rate_0.02_batch_size_64_optimizer_sgd
model_MLPTwo_learning_rate_0.07_batch_size_256_optimizer_sgd
model_MLPTwo_learning_rate_0.02_batch_size_256_optimizer_sgd
model_MLPTwo_learning_rate_0.07_batch_size_64_optimizer_sgd


In [32]:
for m in models:
    print(f"Current model: {m}")
    model = MLPs.MLPTwo()#.to(device)
    evaluation.loadModel(model, m);
    evaluation.validate(model, F.cross_entropy, holdback_loader)

Current model: model_MLPTwo_learning_rate_0.02_batch_size_64_optimizer_sgd

HoldBackSet: Avg. loss: 0.0003, Accuracy: 0.887 (88.7%)

Current model: model_MLPTwo_learning_rate_0.07_batch_size_256_optimizer_sgd

HoldBackSet: Avg. loss: 0.0003, Accuracy: 0.8682 (86.8%)

Current model: model_MLPTwo_learning_rate_0.02_batch_size_256_optimizer_sgd

HoldBackSet: Avg. loss: 0.0001, Accuracy: 0.977 (97.7%)

Current model: model_MLPTwo_learning_rate_0.07_batch_size_64_optimizer_sgd

HoldBackSet: Avg. loss: 0.0001, Accuracy: 0.9813 (98.1%)



In [36]:
src=f'./models/saved/{models[3]}'
new_name = "MLPTwo"
dst=f'./models/saved/{new_name}'
#os.rename(src,dst)

FileNotFoundError: [Errno 2] No such file or directory: './models/saved/model_MLPTwo_learning_rate_0.07_batch_size_64_optimizer_sgd' -> './models/saved/MLPTwo'